In [1]:
# Classificação Multiclasse com SVM - Prevendo Gastos com Cartão de Crédito em 3 Categorias

# Obs: Caso tenha problemas com a acentuação, consulte este link:
# https://support.rstudio.com/hc/en-us/articles/200532197-Character-Encoding

# Definindo o diretório de trabalho
getwd()

[1] "C:/Users/cassi/OneDrive/Programação/DSA/MachineLearning2.0/Cap11-R/R"

In [2]:
####  Definido o Problema de Negócio #### 

# A identificação e a capacidade de classificar os clientes com base nos gastos sempre foram uma área de 
# interesse para instituições bancárias e empresas de cartão de crédito. É um aspecto importante no 
# gerenciamento de relacionamento com o cliente e ajuda a aumentar a receita com clientes existentes. Várias 
# tentativas foram feitas a esse respeito. Os emissores de cartões de crédito tradicionalmente têm como alvo 
# os consumidores usando informações sobre seus comportamentos e dados demográficos. 

# Nosso trabalho é classificar os clientes de cartão de crédito de acordo com seu comportamento de gastos. 
# A segmentação é um aspecto importante na compreensão do cliente e na execução de campanhas de marketing 
# eficazes e rentáveis. Usaremos o SVM como nosso modelo.

# Os dados demográficos, os detalhes sobre emprego e o estilo de vida dos clientes desempenham um papel vital na 
# maneira como eles gastam. Existem fatores ocultos, bem como semelhança com as compras. A máquina de vetores 
# de suporte pode ser usada para problemas de regressão e classificação. 

# Usaremos SVM com Kernel Linear Multiclasse como nosso modelo proposto para classificar a variável target. 
# No entanto, também avaliaremos outros Kernels, como RBF e Polinomial, para uma variedade de hiperparâmetros. 
# Também levamos em consideração o viés no dados.

# Fonte dos dados: https://sorry.vse.cz/~berka/ (dados anônimos)

In [3]:
# Pacotes
install.packages("gains")
install.packages("pROC")
install.packages("ROSE")
install.packages("mice")
library(dplyr)
library(caret)
library(pROC)
library(e1071)
library(mice)
library(readr)

package 'gains' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\cassi\AppData\Local\Temp\RtmpoPDhVV\downloaded_packages
package 'pROC' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'pROC'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\cassi\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\pROC\libs\x64\pROC.dll to C:\Users\cassi\anaconda3\envs\r-tutorial\Lib\R\library\pROC\libs\x64\pROC.dll: Permission denied"Warning message:
"restored 'pROC'"


The downloaded binary packages are in
	C:\Users\cassi\AppData\Local\Temp\RtmpoPDhVV\downloaded_packages
package 'ROSE' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\cassi\AppData\Local\Temp\RtmpoPDhVV\downloaded_packages
package 'mice' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'mice'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\cassi\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\mice\libs\x64\mice.dll to C:\Users\cassi\anaconda3\envs\r-tutorial\Lib\R\library\mice\libs\x64\mice.dll: Permission denied"Warning message:
"restored 'mice'"


The downloaded binary packages are in
	C:\Users\cassi\AppData\Local\Temp\RtmpoPDhVV\downloaded_packages


Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Warning message:
"package 'pROC' was built under R version 3.6.3"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var

Warning message:
"package 'e1071' was built under R version 3.6.3"Warning message:
"package 'mice' was built under R version 3.6.3"
Attaching package: 'mice'

The following objects are masked from 'package:base':

    cbind, rbind



In [5]:
# Carregando os dados pré-processados
?read.csv
dados_treino1 <- read.csv("dados/dados_treino_balanceados.csv")
dados_teste1 <- read.csv("dados/dados_teste.csv")
dim(dados_treino1)
dim(dados_teste1)
head(dados_treino1)
head(dados_teste1)

[1] 8554  118

[1] 1000  118

X,age,ed,union,employ,income,debtinc,creddebt,lncreddebt,othdebt,...,addresscat,cartype,carvalue,carbought,card2,card2type,card2benefit,bfast,internet,Customer_cat
761,18,13,0,0,19,13.0,1.59,0.46,0.88,...,1,0,9.0,0,3,4,4,3,0,medium_spend_cust
2852,20,15,0,0,42,5.3,0.67,-0.41,1.56,...,1,0,19.4,1,2,3,1,1,0,medium_spend_cust
4869,19,14,0,0,18,1.2,0.04,-3.15,0.17,...,1,0,9.4,1,1,3,1,3,3,medium_spend_cust
1393,35,15,0,7,30,2.7,0.25,-1.37,0.56,...,2,1,11.0,1,2,1,1,3,4,medium_spend_cust
3877,43,17,0,8,115,19.8,8.13,2.10,14.64,...,4,1,50.8,0,3,2,4,2,0,low_spend_cust
2609,48,17,0,15,80,3.4,1.52,0.42,1.20,...,4,0,48.0,0,4,4,1,2,0,low_spend_cust


X,age,ed,union,employ,income,debtinc,creddebt,lncreddebt,othdebt,...,addresscat,cartype,carvalue,carbought,card2,card2type,card2benefit,bfast,internet,Customer_cat
2,22,17,0,0,15,18.6,1.22,0.20,1.57,...,1,1,6.8,0,4,1,3,1,4,low_spend_cust
3,67,14,0,16,35,9.9,0.93,-0.07,2.54,...,5,1,18.8,0,4,1,3,3,0,low_spend_cust
6,64,17,0,22,107,5.6,1.06,0.06,4.93,...,5,-1,-1.0,-1,3,3,2,3,0,medium_spend_cust
8,44,16,0,11,97,14.4,5.95,1.78,8.02,...,4,0,55.5,0,3,1,4,3,0,medium_spend_cust
11,59,19,0,8,47,8.6,1.36,0.31,2.68,...,3,0,28.0,0,1,1,4,3,3,medium_spend_cust
15,72,20,1,27,17,9.8,1.28,0.24,0.39,...,3,1,9.3,0,4,4,3,3,0,medium_spend_cust


In [6]:
# A função read_csv mostra o que aconteceu
?read_csv
dados_treino <- read_csv("dados/dados_treino_balanceados.csv")
dados_teste <- read_csv("dados/dados_teste.csv")
dim(dados_treino)
dim(dados_teste)
head(dados_treino)
head(dados_teste)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  .default = col_double(),
  birthmonth = col_character(),
  Customer_cat = col_character()
)
See spec(...) for full column specifications.
Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  .default = col_double(),
  birthmonth = col_character(),
  Customer_cat = col_character()
)
See spec(...) for full column specifications.


[1] 8554  118

[1] 1000  118

X1,age,ed,union,employ,income,debtinc,creddebt,lncreddebt,othdebt,...,addresscat,cartype,carvalue,carbought,card2,card2type,card2benefit,bfast,internet,Customer_cat
761,18,13,0,0,19,13.0,1.59,0.46,0.88,...,1,0,9.0,0,3,4,4,3,0,medium_spend_cust
2852,20,15,0,0,42,5.3,0.67,-0.41,1.56,...,1,0,19.4,1,2,3,1,1,0,medium_spend_cust
4869,19,14,0,0,18,1.2,0.04,-3.15,0.17,...,1,0,9.4,1,1,3,1,3,3,medium_spend_cust
1393,35,15,0,7,30,2.7,0.25,-1.37,0.56,...,2,1,11.0,1,2,1,1,3,4,medium_spend_cust
3877,43,17,0,8,115,19.8,8.13,2.10,14.64,...,4,1,50.8,0,3,2,4,2,0,low_spend_cust
2609,48,17,0,15,80,3.4,1.52,0.42,1.20,...,4,0,48.0,0,4,4,1,2,0,low_spend_cust


X1,age,ed,union,employ,income,debtinc,creddebt,lncreddebt,othdebt,...,addresscat,cartype,carvalue,carbought,card2,card2type,card2benefit,bfast,internet,Customer_cat
2,22,17,0,0,15,18.6,1.22,0.20,1.57,...,1,1,6.8,0,4,1,3,1,4,low_spend_cust
3,67,14,0,16,35,9.9,0.93,-0.07,2.54,...,5,1,18.8,0,4,1,3,3,0,low_spend_cust
6,64,17,0,22,107,5.6,1.06,0.06,4.93,...,5,-1,-1.0,-1,3,3,2,3,0,medium_spend_cust
8,44,16,0,11,97,14.4,5.95,1.78,8.02,...,4,0,55.5,0,3,1,4,3,0,medium_spend_cust
11,59,19,0,8,47,8.6,1.36,0.31,2.68,...,3,0,28.0,0,1,1,4,3,3,medium_spend_cust
15,72,20,1,27,17,9.8,1.28,0.24,0.39,...,3,1,9.3,0,4,4,3,3,0,medium_spend_cust


In [7]:
# Removemos a coluna X criada na indexação randômica
dados_treino <- dados_treino[-1]
dados_teste <- dados_teste[-1]

dim(dados_treino)
dim(dados_teste)

[1] 8554  117

[1] 1000  117

In [8]:
# Transformando a variável target em valor numérico
head(dados_treino)

age,ed,union,employ,income,debtinc,creddebt,lncreddebt,othdebt,lnothdebt,...,addresscat,cartype,carvalue,carbought,card2,card2type,card2benefit,bfast,internet,Customer_cat
18,13,0,0,19,13.0,1.59,0.46,0.88,-0.13,...,1,0,9.0,0,3,4,4,3,0,medium_spend_cust
20,15,0,0,42,5.3,0.67,-0.41,1.56,0.44,...,1,0,19.4,1,2,3,1,1,0,medium_spend_cust
19,14,0,0,18,1.2,0.04,-3.15,0.17,-1.75,...,1,0,9.4,1,1,3,1,3,3,medium_spend_cust
35,15,0,7,30,2.7,0.25,-1.37,0.56,-0.59,...,2,1,11.0,1,2,1,1,3,4,medium_spend_cust
43,17,0,8,115,19.8,8.13,2.10,14.64,2.68,...,4,1,50.8,0,3,2,4,2,0,low_spend_cust
48,17,0,15,80,3.4,1.52,0.42,1.20,0.19,...,4,0,48.0,0,4,4,1,2,0,low_spend_cust


In [9]:
str(dados_treino$Customer_cat)

 chr [1:8554] "medium_spend_cust" "medium_spend_cust" "medium_spend_cust" ...


In [10]:
head(dados_treino$Customer_cat)

[1] "medium_spend_cust" "medium_spend_cust" "medium_spend_cust"
[4] "medium_spend_cust" "low_spend_cust"    "low_spend_cust"

In [18]:
dados_treino$Customer_cat <- as.numeric(as.factor(dados_treino$Customer_cat))
str(dados_treino$Customer_cat)
head(dados_treino$Customer_cat)

 num [1:8554] 3 3 3 3 2 2 2 2 2 2 ...


[1] 3 3 3 3 2 2

In [19]:
dados_treino$Customer_cat <- as.factor(dados_treino$Customer_cat)
str(dados_treino$Customer_cat)
head(dados_treino$Customer_cat)

 Factor w/ 3 levels "1","2","3": 3 3 3 3 2 2 2 2 2 2 ...


[1] 3 3 3 3 2 2
Levels: 1 2 3

In [16]:
dados_teste$Customer_cat <- as.numeric(as.factor(dados_teste$Customer_cat))
dados_teste$Customer_cat <- as.factor(dados_teste$Customer_cat)
str(dados_teste$Customer_cat)
head(dados_teste$Customer_cat)

 Factor w/ 3 levels "1","2","3": 2 2 3 3 3 3 3 3 2 3 ...


[1] 2 2 3 3 3 3
Levels: 1 2 3

In [20]:
##### Modelagem Preditiva ##### 

# Primeira versão do modelo SVM - Versão Padrão com Kernel Radial (RBF)
# O algoritmo escolhe o tipo de SVM de acordo com o tipo de dado da variável target
?svm
modelo_v1 <- svm(Customer_cat ~ ., data = dados_treino, na.action = na.omit, scale = TRUE)
summary(modelo_v1)
print(modelo_v1)


Call:
svm(formula = Customer_cat ~ ., data = dados_treino, na.action = na.omit, 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  4032

 ( 2183 1360 489 )


Number of Classes:  3 

Levels: 
 1 2 3





Call:
svm(formula = Customer_cat ~ ., data = dados_treino, na.action = na.omit, 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  4032



In [21]:
# Fazendo previsões com o modelo
previsoes_v1 <- predict(modelo_v1, newdata = dados_teste)

In [22]:
# Matriz de Confusão
?caret::confusionMatrix
caret::confusionMatrix(previsoes_v1, dados_teste$Customer_cat)

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3
         1   5   0   5
         2   0 187  54
         3  18 116 615

Overall Statistics
                                         
               Accuracy : 0.807          
                 95% CI : (0.7811, 0.831)
    No Information Rate : 0.674          
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.5426         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: 1 Class: 2 Class: 3
Sensitivity            0.2174   0.6172   0.9125
Specificity            0.9949   0.9225   0.5890
Pos Pred Value         0.5000   0.7759   0.8211
Neg Pred Value         0.9818   0.8472   0.7649
Prevalence             0.0230   0.3030   0.6740
Detection Rate         0.0050   0.1870   0.6150
Detection Prevalence   0.0100   0.2410   0.7490
Balanced Accuracy      0.6061   0.7698   0.7507

In [23]:
# Por que o erro aconteceu? Vamos checar! Isso chama-se troubleshooting.

# Comprimento do valor real e do valor previsto
length(dados_teste$Customer_cat)
length(previsoes_v1)

[1] 1000

[1] 1000

In [24]:
# Temos valores NA em teste?
sum(is.na(dados_teste))

[1] 0

In [25]:
# Removemos valores NA
dados_teste = na.omit(dados_teste)
length(dados_teste$Customer_cat)
sum(is.na(dados_teste))

[1] 1000

[1] 0

In [26]:
# E agora sim a matriz de confusão
caret::confusionMatrix(previsoes_v1, dados_teste$Customer_cat)

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3
         1   5   0   5
         2   0 187  54
         3  18 116 615

Overall Statistics
                                         
               Accuracy : 0.807          
                 95% CI : (0.7811, 0.831)
    No Information Rate : 0.674          
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.5426         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: 1 Class: 2 Class: 3
Sensitivity            0.2174   0.6172   0.9125
Specificity            0.9949   0.9225   0.5890
Pos Pred Value         0.5000   0.7759   0.8211
Neg Pred Value         0.9818   0.8472   0.7649
Prevalence             0.0230   0.3030   0.6740
Detection Rate         0.0050   0.1870   0.6150
Detection Prevalence   0.0100   0.2410   0.7490
Balanced Accuracy      0.6061   0.7698   0.7507

In [27]:
# Métricas
install.packages("multiROC")
library(multiROC)

package 'multiROC' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\cassi\AppData\Local\Temp\RtmpoPDhVV\downloaded_packages


Warning message:
"package 'multiROC' was built under R version 3.6.3"

In [28]:
?multiclass.roc
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = previsoes_v1)
class(dados_teste$Customer_cat)
class(previsoes_v1)

ERROR: Error in roc.default(response, predictor, levels = X, percent = percent, : Predictor must be numeric or ordered.


In [29]:
# Faz a conversão
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v1)))

Setting direction: controls > cases
Setting direction: controls < cases
Setting direction: controls < cases


In [30]:
# Score AUC (Area Under The Curve)
curva_roc$auc

# Juntando valores reais e previstos na mesma tabela

# Previsões
valores_previstos <- data.frame(as.numeric(as.factor(previsoes_v1)))
colnames(valores_previstos) <- ("previsão")

# Valores reais
valores_reais <- data.frame(as.numeric(as.factor(dados_teste$Customer_cat)))
colnames(valores_reais) <- ("valor_real")

# Dataframe final
final_df <- cbind(valores_reais, valores_previstos)
View(final_df)


# Segunda versão do modelo SVM - Versão com Kernel Linear e GridSearch

# Vamos fazer uma pesquisa em grade (Grid Search) para o ajuste de hiperparâmetros e usar Kernel linear. 
# Mas aqui não manteremos o custo superior a 2, para que valores discrepantes não afetem extensivamente 
# a criação de limites de decisão e, portanto, levem ao ajuste excessivo (overfitting).
set.seed(182)
?tune
modelo_grid1 <- tune(svm, 
                     Customer_cat ~ ., 
                     data = dados_treino, 
                     kernel = 'linear',
                     ranges = list(cost = c(0.05, 0.1, 0.5, 1, 2))) 


summary(modelo_grid1)

# Parâmetros do melhor modelo
modelo_grid1$best.parameters

# Melhor modelo
modelo_grid1$best.model 
modelo_v2 <- modelo_grid1$best.model 
summary(modelo_v2)

# Previsões
previsoes_v2 <- predict(modelo_v2, dados_teste)

# Matriz de Confusão e Score AUC
confusionMatrix(previsoes_v2, dados_teste$Customer_cat)
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v2)))
curva_roc$auc

# Está oferecendo um desempenho muito melhor em termos de sensibilidade na previsão de classes minoritárias


# Terceira versão do modelo SVM - Versão com Kernel RBF e Otimização no Parâmetro Gamma

# Vamos fazer uma pesquisa em grade para o ajuste de parâmetros com kernel radial, e não manteremos 
# o custo superior a 2, para que os discrepantes não afetem extensivamente a criação de limites 
# de decisão e, portanto, levem ao ajuste excessivo.

# Da mesma forma, não manteremos um valor muito abaixo de 0,001 para gamma, pois isso levaria a 
# um excesso de ajuste.
set.seed(182)
modelo_grid2 <- tune(svm,
                     Customer_cat ~ .,
                     data = dados_treino,
                     kernel='radial',
                     ranges = list(cost = c(0.01,0.05,0.1,0.5,1,2),
                                   gamma = c(0.0001,0.001,0.01,.05,0.1,0.01,1,2)))

summary(modelo_grid2)

# Parâmetros do melhor modelo
modelo_grid2$best.parameters

# Melhor modelo
modelo_v3 <- modelo_grid2$best.model 

# Previsões
previsoes_v3 <- predict(modelo_v3, dados_teste)

# Matriz de Confusão e Score AUC
confusionMatrix(previsoes_v3, dados_teste$Customer_cat)
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v3)))
curva_roc$auc


# Quarta versão do modelo SVM - Versão com Kernel Polinomial

# Vamos fazer uma pesquisa em grade para ajustar parâmetros com kernel poliinomial, e não manteremos 
# o custo superior a 2, para que os discrepantes não afetem extensivamente a criação de limites 
# de decisão e, portanto, levem a um excesso de ajuste.

# Da mesma forma, não manteremos o grau polinomial de ordem superior a 4, pois isso levaria a 
# um ajuste excessivo
set.seed(182)
modelo_grid3 <- tune(svm,
                     Customer_cat ~ .,
                     data = dados_treino,
                     kernel = 'polynomial',
                     ranges = list(cost = c(1,2), degree = c(2,3,4)))

summary(modelo_grid3)

# Parâmetros do melhor modelo
modelo_grid3$best.parameters

# Melhor modelo
modelo_v4 <- modelo_grid3$best.model 

# Previsões
previsoes_v4 <- predict(modelo_v4, dados_teste)

# Matriz de Confusão e Score AUC
confusionMatrix(previsoes_v4, dados_teste$Customer_cat)
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v4)))
curva_roc$auc

# Podemos ver que o modelo ajustado com kernel polinomial tem uma sensibilidade fraca na 
# previsão de clientes com alto gasto para o conjunto de testes.


# Comparação dos Modelos

# Resultados do Modelo 1
resultados_v1 <- caret::confusionMatrix(previsoes_v1, dados_teste$Customer_cat)
resultados_v1$overall
resultados_v1$byClass

# Medidas Globais do Modelo 1
acuracia_v1 <- resultados_v1$overall['Accuracy']
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v1)))
score_auc_v1 <- curva_roc$auc

# Exemplo: Caso você queira outras medidas como precision e recall, lembre-se que elas são por classe
precision_v1_classe1 <- resultados_v1$byClass[1, 'Precision']   
precision_v1_classe2 <- resultados_v1$byClass[2, 'Precision']  
recall_v1_classe3 <- resultados_v1$byClass[3, 'Sensitivity']

# Vetor com os resultados de avaliação do Modelo v1
vetor_modelo1 <- c("Modelo1 Kernel RBF", round(acuracia_v1, 4), round(score_auc_v1, 4))


# Medidas Globais do Modelo 2
resultados_v2 <- caret::confusionMatrix(previsoes_v2, dados_teste$Customer_cat)
acuracia_v2 <- resultados_v2$overall['Accuracy']
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v2)))
score_auc_v2 <- curva_roc$auc

# Vetor com os resultados de avaliação do Modelo v2
vetor_modelo2 <- c("Modelo2 Kernel Linear", round(acuracia_v2, 4), round(score_auc_v2, 4))


# Medidas Globais do Modelo 3
resultados_v3 <- caret::confusionMatrix(previsoes_v3, dados_teste$Customer_cat)
acuracia_v3 <- resultados_v3$overall['Accuracy']
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v3)))
score_auc_v3 <- curva_roc$auc

# Vetor com os resultados de avaliação do Modelo v1
vetor_modelo3 <- c("Modelo3 Kernel RBF Tunning", round(acuracia_v3, 4), round(score_auc_v3, 4))


# Medidas Globais do Modelo 4
resultados_v4 <- caret::confusionMatrix(previsoes_v4, dados_teste$Customer_cat)
acuracia_v4 <- resultados_v4$overall['Accuracy']
curva_roc <- multiclass.roc(response = dados_teste$Customer_cat, predictor = as.numeric(as.factor(previsoes_v4)))
score_auc_v4 <- curva_roc$auc

# Vetor com os resultados de avaliação do Modelo v1
vetor_modelo4 <- c("Modelo4 Kernel Polinomial", round(acuracia_v4, 4), round(score_auc_v4, 4))


# Concatenando os resultados
# Dataframe para os resultados dos modelos
?base::rbind
compara_modelos <- rbind(vetor_modelo1, vetor_modelo2, vetor_modelo3, vetor_modelo4)
View(compara_modelos)
rownames(compara_modelos) <- c("1", "2", "3", "4")
colnames(compara_modelos) <- c("Modelo", "Acuracia", "AUC")
View(compara_modelos)
class(compara_modelos)
compara_modelos <- as.data.frame(compara_modelos)
class(compara_modelos)
View(compara_modelos)

# Plot
library(ggplot2)

# Acurácia
ggplot(compara_modelos, aes(x = Modelo, y = Acuracia, fill = Modelo)) + 
  geom_bar(stat = "identity") 

# AUC
ggplot(compara_modelos, aes(x = Modelo, y = AUC, fill = Modelo)) + 
  geom_bar(stat = "identity")

# Assim, o método final proposto é baseado no Kernel Linear, a versão 2 do nosso modelo.


# Previsões com o Modelo Escolhido

# Salvando o modelo selecionado
?saveRDS
saveRDS(modelo_v2, "modelos/modelo_v2.rds")

# Carregando o modelo salvo
modelo_svm <- readRDS("modelos/modelo_v2.rds")
print(modelo_svm)

# Carrega o arquivo com dados de novos clientes.
# Para esses clientes não temos a variável target, pois isso é o que queremos prever.
novos_clientes <- read.csv("dados/novos_clientes.csv", header = TRUE)
View(novos_clientes)
dim(novos_clientes)

# Fazendo previsões
previsoes_novos_clientes <- predict(modelo_svm, novos_clientes)

# Apresentando o resultado final

# Previsões
previsoes_gastos_novos_clientes <- data.frame(as.numeric(as.factor(previsoes_novos_clientes)))
colnames(previsoes_gastos_novos_clientes) <- ("Previsão de Gasto")

# Idade dos clientes
idades_novos_clientes <- data.frame(novos_clientes$age)
colnames(idades_novos_clientes) <- ("Idades")

# Dataframe final
resultado_final <- cbind(idades_novos_clientes, previsoes_gastos_novos_clientes)
View(resultado_final)

# Ajusta o label da previsão
library(plyr)
?mapvalues
resultado_final$`Previsão de Gasto` <- mapvalues(resultado_final$`Previsão de Gasto`,
                                                 from = c(1,2,3),
                                                 to = c("Alto", "Médio", "Baixo"))

View(resultado_final)
write.csv(resultado_final, "dados/resultado_final.csv")




Multi-class area under the curve: 0.6563

ERROR: Error in View(final_df): 'View()' not yet supported in the Jupyter R kernel


[1] 3 3 3 3 2 2
Levels: 1 2 3